In [133]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import plotly.express as px
import plotly.graph_objects as go

%matplotlib inline

In [134]:
df = pd.read_csv("./data/FAO coffee bean production/coffee-bean-production.csv")
#df.drop(columns="Unnamed: 0",inplace=True)

In [135]:
df.columns = ["Entity","Code","Year","Production"]

In [136]:
china_production = df.loc[df['Entity'] == 'China', 'Production']
taiwan_production = df.loc[df['Entity'] == 'Taiwan', ['Year', 'Production']]
china_year_mask = df['Year'].isin(taiwan_production['Year'])
df.loc[china_year_mask & (df['Entity'] == 'China'), 'Production'] += taiwan_production.loc[taiwan_production['Year'].isin(df.loc[china_year_mask, 'Year']), 'Production'].values

In [137]:
df.loc[df['Entity'] == 'China', 'Production']

1007      1612.00
1008      1756.00
1009      2042.00
1010      2346.00
1011      2577.00
          ...    
1063    115110.14
1064    116519.00
1065    121012.00
1066    114970.00
1067    108906.00
Name: Production, Length: 61, dtype: float64

In [138]:
df = df.drop(index=df[df['Entity'].isin(['Taiwan'])].index)
delete_name = [name for name in df.Entity.unique() if '(FAO)' in name]
continent = ['Africa', 'Asia', 'Oceania', 'Americas', 'Europe']
df = df.drop(index=df[df['Entity'].isin(delete_name)].index)
df = df.drop(index=df[df['Entity'].isin(continent)].index)
mask = df.isna().any(axis=1)
no_code = list(df[mask].Entity.unique())
df = df.drop(index=df[df['Entity'].isin(no_code)].index)

In [139]:
df

,Entity,Code,Year,Production
183,Angola,AGO,1961,168600.00
184,Angola,AGO,1962,185000.00
185,Angola,AGO,1963,168300.00
186,Angola,AGO,1964,198200.00
187,Angola,AGO,1965,205000.00
...,...,...,...,...
7091,Zimbabwe,ZWE,2017,683.33
7092,Zimbabwe,ZWE,2018,627.78
7093,Zimbabwe,ZWE,2019,670.37
7094,Zimbabwe,ZWE,2020,660.49


In [141]:
import plotly.graph_objects as go
import plotly.express as px
import plotly.io as pio

fig = go.Figure()


years = df['Year'].unique()
colors = ["#527A18","#678D38", "#E9BA4E", "#BF8C31", "#6E3D19"]

custom_scale = [[i/(len(colors)-1), colors[i]] for i in range(len(colors))]

# ADD A CHOROPLETH TRACES FOR THE DATA-FRAME
# use a double loop to iterate each continent and for every year, add a trace.
for year in years:
    fig.add_trace(  # Add A trace to the figure
        go.Choropleth(  # Specify the type of the trace
            uid="full-set",  # uid=unique id (Assign an ID to the trace)
            locations=df[(df["Year"] == year)]["Code"],  # Supply location information tag for mapping
            z=np.log(df[(df["Year"] == year)]["Production"]),  # Data to be color-coded on graph
            colorbar_title="Production",  # Title for color-bar
            visible=False,  # Specify whether or not to make data-visible when rendered
            zmin=0,  # Minimum value for the color scale
            zmax=np.log(5000000),  # Maximum value for the color scale
            colorscale=custom_scale,
            hovertemplate="%{location}<br>Production: %{customdata:,}<extra></extra> tons", # Modify hover text to show original values
            customdata=df[(df["Year"] == year)]["Production"]
        )
    )
        

# now, use another double for loop to add slider and button
sliders = []
i = 0

steps = []
for year in years:
    step = dict(
        method="update",
        args=[{"visible":[False]*len(fig.data)}],  # layout attribute
        label = f'{year}'
    )
    step["args"][0]["visible"][i] = True  # Toggle i'th trace to "visible"
    i+=1
    steps.append(step)
slider = [dict(
    active = 0,
    currentvalue={"prefix": "Year: "},
    steps = steps
)]
sliders.append(slider)
# updatemenus = [dict(
#     buttons = buttons,
#     direction="down",
#     showactive=True,  # HIGHLIGHTS ACTIVE DROPDOWN ITEM OR ACTIVE BUTTON IF TRUE
#     pad={"r": 10, "t": 10},  # PADDING
#     x= 0.00,  # POSITION
#     y= 1.5,
#     xanchor="left",  # ANCHOR POINT
#     yanchor="top"
# )] 
fig.data[0].visible = True
# update the layout
fig.update_layout(sliders=sliders[0])
# SHOW
fig.show()